# 哲学理念

🤗 Transformers 是一个具有明确目标的库，致力于服务以下几类人：

- 机器学习研究人员和教育工作者，他们希望使用、研究或扩展大规模的 Transformer 模型。
- 实践者，他们希望微调这些模型或在生产环境中部署这些模型，或者两者兼而有之。
- 工程师，他们只想下载预训练模型并用它来解决特定的机器学习任务。

该库的设计基于两个核心目标：

1. **尽可能简单易用**：

   - 我们尽量减少用户需要学习的抽象概念。实际上，使用每个模型只需了解三个标准类：
     - [配置类](main_classes/configuration)：存储模型的超参数。
     - [模型类](main_classes/model)：实现具体的模型。
     - 预处理类：根据任务不同，分为文本的[分词器](main_classes/tokenizer)、图像的[图像处理器](main_classes/image_processor)、音频的[特征提取器](main_classes/feature_extractor)以及多模态输入的[处理器](main_classes/processors)。
   
   - 所有这些类都可以通过一个通用的 `from_pretrained()` 方法从预训练实例简单且统一地初始化。这个方法会从[Hugging Face Hub](https://huggingface.co/models)或用户自己的保存点下载、缓存并加载相关的类实例和关联数据（如配置的超参数、分词器的词汇表和模型的权重）。
   
   - 在这三个基础类之上，库提供了两个API：
     - [pipeline()](/docs/transformers/v4.47.1/en/main_classes/pipelines#transformers.pipeline)：用于快速在特定任务上进行推理。
     - [Trainer](/docs/transformers/v4.47.1/en/main_classes/trainer#transformers.Trainer)：用于快速训练或微调 PyTorch 模型（所有 TensorFlow 模型都兼容 `Keras.fit`）。
   
   - 因此，此库并不是一个用于构建神经网络的模块化工具箱。如果你想扩展或基于此库构建新功能，可以使用常规的 Python、PyTorch、TensorFlow 或 Keras 模块，继承库的基础类并重用模型加载和保存等功能。如果你想了解有关模型编码哲学的更多信息，可以查看我们的 [重复你自己](https://huggingface.co/blog/transformers-design-philosophy) 博客文章。

2. **提供尽可能接近原版模型的最先进模型**：

   - 我们为每种架构至少提供一个示例，以重现官方作者提供的结果。
   - 代码尽可能接近原始代码库，这意味着一些 PyTorch 代码可能不如原生 PyTorch 代码“地道”，因为它们是从 TensorFlow 代码转换而来的，反之亦然。

其他目标还包括：

- **尽可能一致地暴露模型的内部结构**：
  
  - 使用单一API提供对完整隐藏状态和注意力权重的访问。
  - 标准化预处理类和基础模型API，以便轻松切换模型。

- **集成有前景的微调和研究工具**：
  
  - 提供简单一致的方法来添加新词汇和嵌入以进行微调。
  - 提供简单方法来屏蔽和修剪 Transformer 头。

- **轻松切换 PyTorch、TensorFlow 2.0 和 Flax**，允许使用一个框架进行训练，用另一个框架进行推理。

## 核心概念

该库围绕每个模型的三类类构建：

- **模型类**：可以是 PyTorch 模型([torch.nn.Module](https://pytorch.org/docs/stable/nn.html#torch.nn.Module))、Keras 模型([tf.keras.Model](https://www.tensorflow.org/api_docs/python/tf/keras/Model)) 或 JAX/Flax 模型([flax.linen.Module](https://flax.readthedocs.io/en/latest/api_reference/flax.linen/module.html))，它们使用库中提供的预训练权重。

- **配置类**：存储构建模型所需的超参数（如层数、隐藏单元大小）。如果使用的是预训练模型而没有修改，创建模型时会自动实例化配置，因此你不一定需要手动实例化这些类。

- **预处理类**：将原始数据转换为模型可接受的格式。[分词器](main_classes/tokenizer)存储每个模型的词汇表，并提供将字符串编码为标记嵌入索引列表的方法。[图像处理器](main_classes/image_processor)预处理视觉输入，[特征提取器](main_classes/feature_extractor)预处理音频输入，[处理器](main_classes/processors)处理多模态输入。

所有这些类都可以通过以下三种方法实例化、保存本地并共享到 Hub：

- `from_pretrained()`：从预训练版本实例化模型、配置和预处理类。预训练模型可以来自库本身支持的模型（可在[模型库](https://huggingface.co/models)中找到），也可以是用户本地保存的模型。
- `save_pretrained()`：将模型、配置和预处理类保存到本地，以便使用 `from_pretrained()` 重新加载。
- `push_to_hub()`：将模型、配置和预处理类共享到 Hub，以便他人轻松访问。